In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
import numpy as np

In [4]:
config=tf.ConfigProto()
config.gpu_options.allow_growth = True

In [5]:
tf.set_random_seed(777)

In [6]:
# parameters
log_dir = './logs/ML_Lab_11-05-CNN'
learning_rate = 0.001
batch_size = 100
num_epochs = 50

In [7]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [8]:
X = tf.placeholder(shape=[None, 784], dtype=tf.float32, name='X')

In [9]:
Y = tf.placeholder(shape=[None,  10], dtype=tf.float32, name='Y')

In [10]:
X_img = tf.reshape(X, shape=[-1, 28, 28, 1])

In [11]:
with tf.variable_scope('Conv1') as scope:
    W1 = tf.Variable(tf.random_normal(shape=[3, 3, 1, 32], stddev=0.01), dtype=tf.float32, name='weight1')
    L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    print(L1)

Tensor("Conv1/MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)


In [12]:
with tf.variable_scope('Conv2') as scope:
    W2 = tf.Variable(tf.random_normal(shape=[3, 3, 32, 64], stddev=0.01), dtype=tf.float32, name='weight2')
    L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    print(L2)

Tensor("Conv2/MaxPool:0", shape=(?, 7, 7, 64), dtype=float32)


In [13]:
with tf.variable_scope('Fuuly-connected') as scope:
    L2 = tf.reshape(L2, shape=[-1, 7 * 7 * 64])
    W3 = tf.get_variable('weight3', shape=[7 * 7 * 64, 10], initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal(shape=[10]), dtype=tf.float32, name='bias3')
    hypothesis = tf.matmul(L2, W3) + b3
    print(hypothesis)

Tensor("Fuuly-connected/add:0", shape=(?, 10), dtype=float32)


In [14]:
with tf.variable_scope('cost'):
    cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y)
    cost = tf.reduce_mean(cost_i)

In [15]:
with tf.variable_scope('Optimizer'):
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [16]:
with tf.variable_scope('accuracy') as scope:
    is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.arg_max(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

Instructions for updating:
Use `tf.math.argmax` instead


In [17]:
with tf.variable_scope('tensorboard') as scope:
    overall_cost     = tf.placeholder(shape=None, dtype=tf.float32, name='overall_cost')
    overall_accuracy = tf.placeholder(shape=None, dtype=tf.float32, name='overall_accuracy')
    tf.summary.scalar('cost', overall_cost)
    tf.summary.scalar('accuracy', overall_accuracy)

In [18]:
with tf.Session(config=config) as sess:
    # tensorboard --logdir=./logs/Deep_NN
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(log_dir)
    writer.add_graph(sess.graph)
    
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    # Training cycle
    for epoch in range(num_epochs):
        _overall_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            cost_val, _ = sess.run([cost, train], feed_dict={X: batch_xs, Y: batch_ys})
            _overall_cost += cost_val / total_batch
        
        _overall_accuracy = 0
        total_batch = int(mnist.test.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.test.next_batch(batch_size)
            accuracy_val = sess.run(accuracy, feed_dict={X: batch_xs, Y: batch_ys})
            _overall_accuracy += accuracy_val / total_batch
            
        summary = sess.run(merged_summary, feed_dict={overall_cost: _overall_cost, overall_accuracy: _overall_accuracy})
        writer.add_summary(summary, global_step=(epoch))
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(_overall_cost), 'accuracy =', '{:.9f}'.format(_overall_accuracy))

Epoch: 0001 cost = 0.345581816 accuracy = 0.970700007
Epoch: 0002 cost = 0.089809411 accuracy = 0.974200010
Epoch: 0003 cost = 0.067674202 accuracy = 0.982500012
Epoch: 0004 cost = 0.056645692 accuracy = 0.982100009
Epoch: 0005 cost = 0.047604032 accuracy = 0.986100010
Epoch: 0006 cost = 0.041491933 accuracy = 0.986500008
Epoch: 0007 cost = 0.035762655 accuracy = 0.986100009
Epoch: 0008 cost = 0.031863012 accuracy = 0.985000009
Epoch: 0009 cost = 0.028265070 accuracy = 0.987900009
Epoch: 0010 cost = 0.024407292 accuracy = 0.985300011
Epoch: 0011 cost = 0.021742031 accuracy = 0.988800009
Epoch: 0012 cost = 0.019607739 accuracy = 0.989200010
Epoch: 0013 cost = 0.016717939 accuracy = 0.988200010
Epoch: 0014 cost = 0.015167358 accuracy = 0.988800008
Epoch: 0015 cost = 0.013067828 accuracy = 0.987100010
Epoch: 0016 cost = 0.012344268 accuracy = 0.987800010
Epoch: 0017 cost = 0.011103002 accuracy = 0.987500011
Epoch: 0018 cost = 0.008696620 accuracy = 0.987700008
Epoch: 0019 cost = 0.0087569